In [1]:
%load_ext autoreload
%autoreload 2

import os

print("original dir: ", os.getcwd())

new_path = "../"
os.chdir(new_path)

print("changed dir: ", os.getcwd())

original dir:  d:\我的\大学\3秋\学业\创新实践\repo\Nonlinear-Erasure-Code\src\test
changed dir:  d:\我的\大学\3秋\学业\创新实践\repo\Nonlinear-Erasure-Code\src


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from tqdm import tqdm
import datetime
import os

from base_model.LeNet5 import LeNet5

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 数据加载
# 设置数据转换 - 这里只使用基本的转换
transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))]
)

# 加载训练集
train_dataset = datasets.MNIST(
    root="./data", train=True, download=True, transform=transform
)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# 加载测试集
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

"""
train_dataset: [(image, label), (image, label), ...]
len(train_dataset): 60000
image: (1, 28, 28)

test_dataset: [(image, label), (image, label), ...]
len(test_dataset): 10000
image: (1, 28, 28)
"""
print(f"Train dataset: {len(train_dataset)}")
print(f"Test dataset: {len(test_dataset)}")
print("image size: ", train_dataset[0][0].size())

# 定义模型
model = LeNet5(input_dim=(1, 28, 28), num_classes=10)
model.to(device)

# 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


# 训练循环
num_epochs = 10  # 迭代次数
model.train()  # 设置模型为训练模式

for epoch in range(num_epochs):
    # 使用 tqdm 包装训练数据加载器
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
    for data, target in train_loader_tqdm:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        # 正向传播
        output = model(data)
        loss = criterion(output, target)

        # 反向传播和优化
        optimizer.zero_grad()  # 清除之前的梯度
        loss.backward()  # 反向传播
        optimizer.step()  # 更新权重

        # 更新进度条的描述
        train_loader_tqdm.set_postfix(loss=loss.item())
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")


# 测试循环
model.eval()  # 设置模型为评估模式
correct = 0
total = 0

with torch.no_grad():  # 在评估过程中不计算梯度
    for data, target in test_loader:
        # 将数据移动到设备上
        data, target = data.to(device), target.to(device)

        output = model(data)
        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

print(f"Accuracy on the Test set: {100 * correct / total}%")

# 保存模型
now = datetime.datetime.now()
date = now.strftime("%Y_%m_%d")
filepath = f"base_model/LeNet5/MNIST/{date}/model.pth"
dirpath = os.path.dirname(filepath)
if not os.path.exists(dirpath):
    os.makedirs(dirpath)
torch.save(model.state_dict(), filepath)

# 读取模型
# model = LeNet5(input_dim=(1, 28, 28), num_classes=10)
# model.load_state_dict(torch.load(filepath))

# gpu: 3min58s
# cpu: 5min17s

d:\Install_Location\Anaconda\envs\gp_nlec\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Device: cuda


9913344it [00:14, 700324.54it/s]                              


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



29696it [00:00, 19413037.97it/s]         


...

Epoch 8/10, Loss: 0.016005150973796844


Epoch 9/10: 100%|██████████| 938/938 [00:15<00:00, 62.05it/s, loss=0.00402] 


Epoch 9/10, Loss: 0.004020973574370146


Epoch 10/10: 100%|██████████| 938/938 [00:15<00:00, 62.00it/s, loss=0.000775]


Epoch 10/10, Loss: 0.0007751308148726821
Accuracy on the Test set: 99.04%
